In [47]:
from utils import pipeline as pp
import pandas as pd
import glob
cnn=pp.get_conexion()
schema='egresados'
import numpy as np



In [37]:
df_benchmark=pd.read_sql_query('select agno, rbd, cod_ense, mrun from egresados.fact_egresados where egresado_flg=1',cnn)
df_benchmark

,agno,rbd,cod_ense,mrun
0,2011,2219,710,108578
1,2011,2219,710,317502
2,2011,2219,710,557110
3,2011,2219,710,573298
4,2011,2219,710,700528
...,...,...,...,...
22621,2023,25185,310,26951902
22622,2023,25185,310,26966725
22623,2023,25185,310,26988498
22624,2023,25185,310,27423366


# **Pruebas de Admisión**

In [66]:
df_prueba=pd.read_csv('pruebas_admision/A_INSCRITOS_PUNTAJES_PSU_2018_PRIV_MRUN.csv',sep=';',encoding='latin1')
df_prueba.info()
# df_egresados=df_rendimiento.loc[:,
#                    #[i.lower() for i in
#                     ['AGNO', 'RBD', 'COD_ENSE', 'COD_GRADO', 'LET_CUR',  'MRUN',  'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN','SIT_FIN_R']
#                     #]
#                    ]
# df_egresados.to_csv('egresados_2011.csv',sep=';')
#df_prueba.columns
# new_cols=['mrun', 'anyo_proceso','rbd','prom_notas','nem','clec','mate','hcsoc','cien','ranking']
# df_prueba=df_prueba.loc[:,['MRUN', 'AÑO_PROCESO','RBD','PROMEDIO_NOTAS','PTJE_NEM','LYC_ACTUAL','MATE_ACTUAL','HYCS_ACTUAL','CIENCIAS_ACTUAL','PTJE_RANKING']]
# df_prueba.columns=new_cols
# df_prueba.to_csv('prueba_admision_proceso_2012.csv',sep=';')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306376 entries, 0 to 306375
Data columns (total 62 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ï»¿TIPO_IDENTIFICACION  306376 non-null  object
 1   MRUN                    306376 non-null  int64 
 2   AÃO_PROCESO            306376 non-null  int64 
 3   COD_SEXO                306376 non-null  int64 
 4   FECHA_NACIMIENTO        306376 non-null  int64 
 5   RBD                     306376 non-null  int64 
 6   CODIGO_ENSEÃANZA       306376 non-null  int64 
 7   LOCAL_EDUCACIONAL       306376 non-null  int64 
 8   UNIDAD_EDUCATIVA        306376 non-null  int64 
 9   NOMBRE_UNIDAD_EDUC      306376 non-null  object
 10  RAMA_EDUCACIONAL        306376 non-null  object
 11  GRUPO_DEPENDENCIA       306376 non-null  int64 
 12  CODIGO_REGION           306376 non-null  int64 
 13  NOMBRE_REGION           306376 non-null  object
 14  CODIGO_PROVINCIA        306376 non-n

In [79]:
lst_dfs=[]
for file in glob.glob('bases/pruebas_admision/*.csv'):
    lst_dfs.append(pd.read_csv(file,sep=';'))
df_prueba=pd.concat(lst_dfs)
df_prueba['rbd']=pd.to_numeric(df_prueba['rbd'], errors='coerce', downcast='integer')
df_prueba['clec']=pd.to_numeric(df_prueba['clec'], errors='coerce', downcast='integer')
df_prueba['mate']=pd.to_numeric(df_prueba['mate'], errors='coerce', downcast='integer')
df_prueba['hcsoc']=pd.to_numeric(df_prueba['clec'], errors='coerce', downcast='integer')
df_prueba['cien']=pd.to_numeric(df_prueba['mate'], errors='coerce', downcast='integer')
df_prueba.to_csv('prueba_admision_proceso_2011_2023.csv',sep=';')

In [81]:
df_prueba_admision=pd.merge(df_prueba,df_benchmark,how='inner',on=['mrun','rbd']).loc[:,['anyo_proceso','agno', 'rbd', 'cod_ense','mrun',  'prom_notas', 'nem', 'clec', 'mate',
       'hcsoc', 'cien', 'ranking']]
columnas_a_procesar = ['clec', 'mate', 'hcsoc', 'cien']
for columna in columnas_a_procesar:
    df_prueba_admision[columna] = df_prueba_admision[columna].replace(0, np.nan)


In [82]:
tabla='prueba_admision_edsup'
pp.insert_table(cnn,schema, tabla,df_prueba_admision)

La tabla <<egresados.prueba_admision_edsup>> no existe, omitiendo


In [29]:
df_egresados=  pd.merge(df_egresados[df_egresados['COD_GRADO']==4],df_benchmark,how='inner', left_on=['RBD','COD_ENSE'],right_on=['bench_rbd','cod_ense']).loc[:,
                   #[i.lower() for i in
                    ['AGNO', 'RBD', 
       'COD_ENSE', 'COD_GRADO', 'LET_CUR',  'MRUN',  'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN',
       'SIT_FIN_R']]#]
df_egresados.columns=[i.lower() for i in df_egresados.columns]

In [30]:
tabla='egresados'

pp.insert_table(cnn,schema, tabla,df_egresados)

La tabla <<egresados.egresados>> no existe, omitiendo
